In [49]:
#####################################################
# ET-robot-contest Game Area Detection and find block color.
# Copyright © 2022 naoki hunada. All rights reserved.
#####################################################

In [50]:
from sklearn.decomposition import PCA
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

from copy import deepcopy
import sys
import math
import array

import area_detect
import perspective

In [51]:
param = area_detect.option()
ad = area_detect.AreaDetection()

In [52]:
def init():
    param.PROCESS_IMAGE_SIZE = [300, 320]
    param.IMAGE_MASK_MIN_THRESHOLD = [0,51,70] #HSV
    param.IMAGE_MASK_MAX_THRESHOLD = [255,255,255] #HSV
    param.LINE_COLOR_DETECTION_FINENESS = 30
    param.LINE_COLOR_DETECTION_SCORE_THRESHOLD = 550
    param.LINE_COLOR_DETECTION_MAX_THRESHOLD = [255,255,45] #HSV
    param.LINE_DETECTION_ROTATE_RANGE = 1.7
    param.POINT_DETECTION_MIN_AREA_THRESHOLD = 30
    param.POINT_DETECTION_MAX_AREA_THRESHOLD = 450
    param.BLUE_CIRCLE_DETECTION_MIN_THRESHOLD = [50, 0, 0] #HSV
    param.BLUE_CIRCLE_DETECTION_MAX_THRESHOLD = [90, 255, 255] #HSV
    param.GREEN_CIRCLE_DETECTION_MIN_THRESHOLD = [90, 0, 0] #HSV
    param.GREEN_CIRCLE_DETECTION_MAX_THRESHOLD = [125, 255, 255] #HSV

    ad.set_option(param)

init()

In [53]:
### カメラデバイスを使う(リアルタイム)
def camera():
    window_name = "camera_et"
    cap = cv2.VideoCapture(2)

    if not cap.isOpened():
        sys.exit()
    ret, frame = cap.read()
    width = frame.shape[0]
    height = frame.shape[1]
    
    while True:
        #カメラからの画像取得
        ret, frame = cap.read()
        if not ret:
            break
        out_img, point, success = ad.detect(frame)
        img = out_img[0]
        if(success == True):
            img = perspective.area_perspective_transform(img, point, 82)
        img = cv2.resize(img, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))) #(int(width*2), int(height*2))
        #カメラの画像の出力
        cv2.imshow('camera' , img)

        #繰り返し分から抜けるためのif文
        key = cv2.waitKey(33)
        if key == 27:
            break

    #メモリを解放して終了するためのコマンド
    cap.release()
    cv2.destroyAllWindows()

In [54]:
### 動画データを使う
def video(file_name):
    window_name = "video_et"
    cap_file = cv2.VideoCapture(file_name)

    if not cap_file.isOpened():
        sys.exit()
    ret, frame = cap_file.read()
    width = frame.shape[0]
    height = frame.shape[1]
    

    wait_time = int(round(cap_file.get(cv2.CAP_PROP_FPS), 0))
    while True:
        ret, frame = cap_file.read()
        if not ret:
            break
        out_img, point, success = ad.detect(frame)
        img = out_img[0]
        if(success == True):
            s_point = point
        
        img = perspective.area_perspective_transform(img, s_point, 82)
        img = cv2.resize(img, (int(cap_file.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap_file.get(cv2.CAP_PROP_FRAME_HEIGHT)))) #(int(width*1.2), int(height*1.2))
        cv2.imshow(window_name, img)

        key = cv2.waitKey(wait_time)
        if key == 27:
            break

    cap_file.release()
    cv2.destroyWindow(window_name)

In [55]:
###画像データを使う
def image(file_name):
    img = cv2.imread(file_name)
    out_img, point, success = ad.detect(img)
    img = out_img[0]
    if(success == True):
        img = perspective.area_perspective_transform(img, point, 82)
    cv2.imwrite("a.png",np.asarray(img))
    plt.imshow(np.asarray(img))

In [56]:
##### execute #####

#image("pic2.png")
video("video2.mp4")
#camera()